In [ ]:
# Install the python-dotenv package if you haven't already
# !pip install python-dotenv
# !pip install seaborn


In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import seaborn as sns

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [ ]:
# Get the database URL from the environment variable
DATABASE_URL = os.getenv('DATABASE_URL')


In [ ]:
# Create an SQLAlchemy engine
engine = create_engine(DATABASE_URL)


In [ ]:
# Define your start and end date as parameters
start_date = '2024-01-01'  # Replace with your start date
end_date = '2024-02-01'    # Replace with your end date

# SQL query with numbered placeholders for parameters
# Using the SQLAlchemy engine, we must use the format %(name)s for placeholders.
query = """
SELECT
    users.username as user_id,
    DATE_TRUNC('day', events.time) as period,
    COUNT(*) as messages
FROM 
    events
JOIN 
    users ON events.user_username = users.username
WHERE
    events.type = 'user_message'
AND 
    events.time::date >= %(start_date)s
AND 
    events.time::date <= %(end_date)s
GROUP BY
    1, 2
ORDER BY
    2 DESC
"""

# Read the query result into a DataFrame using the parameters
df = pd.read_sql_query(query, engine, params={'start_date': start_date, 'end_date': end_date})

In [ ]:
# Convert 'period' column to string to avoid issues with plotting
df['period'] = df['period'].astype(str)

# Pivot the DataFrame to have periods as the index and user_ids as the columns
pivot_df = df.pivot(index='period', columns='user_id', values='messages')

# Plot a stacked bar chart
pivot_df.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='tab20')

plt.xlabel('Period')
plt.ylabel('Messages')
plt.title('Stacked message count by user over time')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to prevent clipping of labels

# Move the legend outside of the plot area
plt.legend(title='User ID', loc='upper left', bbox_to_anchor=(1, 1))

# Show the plot with the legend moved out of the way
plt.show()